<a href="https://colab.research.google.com/github/himanshunaidu/cnn_image_caption/blob/master/ImageCaption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [1]:
from google.colab import files
uploaded = files.upload()

Saving preprocess.py to preprocess.py


In [71]:
import os
import tensorflow as tf
import numpy as np
import math
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2, inception_v3
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as kb
from tqdm import tqdm

In [3]:
print(tf.__version__)

2.5.0


In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [10]:
!ls

gdrive	preprocess.py  __pycache__  sample_data


# DATASET

In [9]:
from preprocess import loadCaptions, getImages

In [29]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [7]:
image_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

87916544/87910968 [==============================] - 1s 0us/step


In [67]:
base_path = './gdrive/MyDrive/ImageCaptions/'

ds_path = 'Flicker8k_Dataset/'
feature_path = 'Flicker8k_Dataset_Features/'
train_images_path = 'Flickr_8k.trainImages.txt'

In [22]:
img_name_list = getImages(os.path.join(base_path, train_images_path))
print(len(img_name_list))

6001


In [24]:
img_path_list = [os.path.join(base_path, ds_path, filename) for filename in img_name_list]
# print(img_path_list[0])
img_dataset = tf.data.Dataset.from_tensor_slices(img_path_list)

In [30]:
img_dataset = img_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(16)

In [31]:
img_dataset

<BatchDataset shapes: ((None, 299, 299, 3), (None,)), types: (tf.float32, tf.string)>

In [69]:
#Experiment For File Saving
path0 = img_dataset.take(1)
for i, p in path0:
  path_of_feature = p[0].numpy().decode("utf-8")
  print(path_of_feature)
  print(path_of_feature.replace(ds_path, feature_path, 1))
path0

./gdrive/MyDrive/ImageCaptions/Flicker8k_Dataset/2513260012_03d33305cf.jpg
./gdrive/MyDrive/ImageCaptions/Flicker8k_Dataset_Features/2513260012_03d33305cf.jpg


<TakeDataset shapes: ((None, 299, 299, 3), (None,)), types: (tf.float32, tf.string)>

# Model

In [8]:
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

features_model = tf.keras.Model(new_input, hidden_layer)

In [63]:
hidden_layer

<KerasTensor: shape=(None, None, None, 2048) dtype=float32 (created by layer 'mixed10')>

In [ ]:
for img, path in tqdm(img_dataset):
  batch_features = features_model(img)
  batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8").replace(ds_path, feature_path, 1)
    np.save(path_of_feature, bf.numpy())